In [13]:
from typing import TypedDict,Literal,Annotated

from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
groq_api=os.getenv('groq_api')
llm=ChatGroq(model='gemma2-9b-it',api_key=groq_api)
from langgraph.graph import StateGraph,START,END
from pydantic import BaseModel,Field
import operator
from langchain_core.messages import HumanMessage,SystemMessage,BaseMessage
from langgraph.checkpoint.memory import MemorySaver



In [3]:
from langgraph.graph.message import add_messages ## its a reducer function like operator.add it is more optimize to work with base messages

class ChatState(TypedDict):
    message:Annotated[list[BaseMessage],add_messages]    ## it can be list[str] but list[basemessage] is perpect all types of message like system, human, ai all comes under system message

In [4]:
def chat_node(state:ChatState):
    message=state['message']
    response=llm.invoke(message).content
    return {'message':response}

In [ ]:

checkpointer=MemorySaver()
graph=StateGraph(ChatState)

graph.add_node('chat_node',chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

workflow=graph.compile(checkpointer=checkpointer)


In [6]:
init_state={'message':HumanMessage("what is the capital of bihar?")}
reply=workflow.invoke(init_state)

In [7]:
reply

{'message': [HumanMessage(content='what is the capital of bihar?', additional_kwargs={}, response_metadata={}, id='9f97bfef-9e3e-4a84-9dcc-2ec94ccad42d'),
  HumanMessage(content='The capital of Bihar is **Patna**. \n', additional_kwargs={}, response_metadata={}, id='2f058720-2894-4368-af90-4dea9d75906c')]}

In [8]:
reply['message'][-1].content

'The capital of Bihar is **Patna**. \n'

In [9]:
st=input('what is my name?')

In [10]:
st

''

In [15]:
thread_id=1
while True:
    question=input('for exit type end....')
    if(question!='end'):
        init_state={'message':HumanMessage(content=question)}
        config={'configurable':{'thread_id':thread_id}}

        reply=workflow.invoke(init_state,config=config)['message'][-1].content
        print('Humman :', question )
        print('AI :' ,reply)
    else:
        break

Humman : hi my name is sahil
AI Hi Sahil, it's nice to meet you!

What can I do for you today? 😄  

Humman : what is my name
AI Your name is Sahil.  😊 

I remember that you told me!  Is there anything else I can help you with?

Humman : can you add 199+1
AI 200 😄  

Humman : now  multiply with 1o
AI 2000  😄  



In [20]:
workflow.get_state(config=config).values

{'message': [HumanMessage(content='hi my name is sahil', additional_kwargs={}, response_metadata={}, id='af3f05b5-a31c-4180-a24b-a8e9dfa8f244'),
  HumanMessage(content="Hi Sahil, it's nice to meet you!\n\nWhat can I do for you today? 😄  \n", additional_kwargs={}, response_metadata={}, id='0649d2d2-d9ad-4799-a983-80f86ef66484'),
  HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}, id='af59ff75-d9ea-4a88-af5a-ce61945e2b39'),
  HumanMessage(content='Your name is Sahil.  😊 \n\nI remember that you told me!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={}, id='9cc1ff33-6793-4e68-8e22-b9c413ef5ff9'),
  HumanMessage(content='can you add 199+1', additional_kwargs={}, response_metadata={}, id='b5f86eb8-d684-459c-8816-0584d2f2daf6'),
  HumanMessage(content='200 😄  \n', additional_kwargs={}, response_metadata={}, id='fef5df13-7b7e-4699-8129-5f5a4be0c054'),
  HumanMessage(content='now  multiply with 1o', additional_kwargs={